In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
nrows = int(2e5) #cantidad de filas en los datos a ser generados
nit = 300 #número de instantes de tiempo que se intentará reproducir
files = os.listdir('Data') #listado de todos los archivos de texto generados
start = -1 #Inicio de los datos para la interpolación
end = 1 #Fin de los datos para la interpolación
ndiv = int(nrows**(1/3)) #Número de divisiones en cada eje, por eso se obtiene de la raíz cúbica y se trunca
files

In [ ]:
p = [file[12:-4].split('_') for file in files] #Obtención de los puntos por cada nombre de archivo de datos
p = np.array(p) #Transformación de los puntos a numpy ndarray

#Carga de cada dato, se realiza la transposición para convertir en fila y se suma los valores de transmisión directa y difusa
df = pd.DataFrame([pd.read_csv('Data/'+file,header=None,sep='\t').T.sum(axis=0).values for file in files])

df_p = pd.DataFrame() #Dataframe vacío para cargar los puntos obtenidos anteriormente
df_p['px'], df_p['py'], df_p['pz'] = p[:,0], p[:,1], p[:,2] #Carga de los puntos en el dataframe
df_p = df_p.astype('float64') #Transformación de los puntos a flotante
df_p = df_p.values #Obtención de los puntos en forma de numpy ndarray

In [ ]:
coor = np.linspace(start,end,ndiv) #Coordenadas para la división que se va a realizar en cada eje, espacios equidistantes
new_points = np.array(np.meshgrid(coor,coor,coor)).T.reshape(-1,3) #Creación de los nuevos puntos con todas las combinaciones posibles y transposición
nrows = new_points.shape[0] #Obtención de la verdadera cantidad de filas que se van a crear
nrows

In [ ]:
#Obtención de las distancias de cada uno de los nuevos puntos a los puntos anteriores, se repite los nuevos puntos y se forma 
#2 matrices, una con los nuevos puntos y la otra con los puntos originales para luego realizar una resta completa y obtener una
#matriz de distancias de cada eje de cada punto nuevo hacia los puntos originales
dist = np.abs(np.tile(new_points,27)-np.repeat(df_p.reshape(1,-1),nrows,axis=0))

dist_sel = np.round(dist,10) #Redondeo de las distancias obtenidas para corregir valores de 0, 1 y 2

#Selección de todas las filas, y las columnas de 3 en 3 para selecciones por separado las distancias que son mayores a 1 en cada
#eje, así podemos evitar que se hagan interpolaciones con valores que estén muy alejados
ds = (dist_sel[:,::3]>1).astype('int')+(dist_sel[:,1::3]>1).astype('int')+(dist_sel[:,2::3]>1).astype('int')

#Se ordena de menor a mayor, dado que se transformó el booleano en entero se puede utilizar esto para tomar los valores de 0
#como aquellas distancias a los puntos más cercanos
cls = np.argsort(ds,axis=1)[:,:8] #argsort devuelve los índices del nuevo orden

distances = pd.DataFrame() #Se crea un dataframe para almacenar las distancias finales que se usarán para la interpolación

#Selección de las distancias más cercanas, asignándolas a cada nuevo punto según su id y usando selección condicional
for i in range(0,24,3):
    distances[str(i)]=dist[:,::3][np.arange(nrows), cls[:,int(i/3)]]
    distances[str(i+1)]=dist[:,1::3][np.arange(nrows), cls[:,int(i/3)]]
    distances[str(i+2)]=dist[:,2::3][np.arange(nrows), cls[:,int(i/3)]]
mult = (1-distances.abs()).values #Se usa el valor de 1 menos las distancias para la multiplicación de la interpolación

In [ ]:
val = pd.DataFrame() #Se crea un dataframe para los valores que se usarán en las interpolaciones
for i in range(8):
    
    #Se obtiene el valor correspondiente a los 8 puntos más cercanos y en cada punto se multiplican los valores de 'x', 'y', 'z'
    val[str(i)] = np.prod(mult[:,i*3:i*3+3],axis=1) 
    
val = np.repeat(val.values,nit,axis=1) #Se repite el valor del porcentaje de cada punto el número de veces necesaria
cls = pd.DataFrame(cls)

In [ ]:
df

In [ ]:
#Se define una función que creará fila a fila el dataframe final compuesto de los valores interpolados
def getBigDf(x):
    
    #Se recibe el índice, se extraen los valores que le corresponden y se multiplica por los datos según el índice dado por la 
    #fila de la variable cls, se recorre hasta los instantes de tiempo acordados
    minidf = val[x.name] * df.iloc[x,:nit].values.reshape(1,-1)[0] 
    
    retorno = [np.sum(minidf[i::nit]) for i in range(nit)] #Se suman los valores de los 3 ejes para cada instante
    return retorno

#Se aplica la función definida anteriormente a cada fila y se expande el resultado que llega a modo de lista
bigdf = pd.DataFrame(cls.apply(lambda x: getBigDf(x),axis=1,result_type='expand'))

points = pd.DataFrame(new_points) #Se crea un dataframe con los nuevos puntos que fueron creados
points.columns = ['x','y','z']

#Se guarda los datos creados por la interpolación para no tener que crearlos nuevamente
bigdf.to_csv('targets.csv',header=False,index=False,chunksize=10000)
points.to_csv('points.csv',header=False,index=False,chunksize=10000)

In [ ]:
#Ahora ya se puede cargar los datos creados por la interpolación
bigdf = pd.read_csv('targets.csv',header=None)
points = pd.read_csv('points.csv',header=None)

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from time import sleep

In [ ]:
distances

In [ ]:
#Se define una función para graficar el comportamiento de la energía
def graph(x):
    clear_output(wait=True)
    plt.plot(x.index, x)
    coo = points.iloc[x.name].values
    plt.title("x: {}, y: {}, z:{}".format(coo[0],coo[1],coo[2]))
    plt.show()

#Se procede a graficar el comportamiento de la energía de los nuevos puntos para una verificación visual
for i in range(1):
    graph(bigdf.iloc[i])

In [ ]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras import callbacks

In [ ]:
#Se separa los datos en 3 sets usando una semilla para asegurar que se separen en el mismo orden
x_train, x_test = train_test_split(points, train_size=0.85, random_state=5, shuffle=True)
x_val, x_test = train_test_split(x_test, train_size=0.5, random_state=5, shuffle=True)

y_train, y_test = train_test_split(bigdf, train_size=0.85, random_state=5, shuffle=True)
y_val, y_test = train_test_split(y_test, train_size=0.5, random_state=5, shuffle=True)

In [ ]:
#Se pone una semilla a las librerías que se usará para el deep learning
np.random.seed(5)
tf.random.set_seed(5)

In [ ]:
#Se crea el perceptrón usando la api funcional de keras
inp = keras.layers.Input(points.shape[1],name='entrada') #Se crea la entrada

#Dado que la función de activación tanh puede dar valores que crecen indefinidamente, se usa sigmoides después
#Así se controla que los datos no tiendan a infinito fácilmente
x = keras.layers.Dense(15,activation='tanh',name='capa1')(inp) #primera capa incrementando la dimensión de 3 a 15
x = keras.layers.Dense(35,activation='softplus',name='capa2')(x) #segunda capa incrementando a 35 la dimensión
x = keras.layers.Dense(75,activation='sigmoid',name='capa3')(x) #tercera capa incrementando a 75 la activación
x = keras.layers.Dense(150,activation='tanh',name='capa4')(x) #cuarta capa incrementando a 150 la activación

#Para salida se usa relu, dado que los valores están entre 0 y 1
x = keras.layers.Dense(bigdf.shape[1],activation='relu',name='salida')(x)

In [ ]:
#Se crea el modelo indicando entradas y salidas
model = keras.Model(
    inputs = inp,
    outputs = x
)

In [ ]:
#Se compila el modelo con optimizador de rmsprop, que mantiene una media móvil del cuadrado de los gradientes y
#luego divide el nuevo gradiente para la raíz de esta media. La ventaja de este algoritmo es que automáticamente reduce
#el tamaño de los pasos que se toman a medida que se aproxima al mínimo. Se utiliza el error medio cuadrático para medir
#la pérdida y una métrica de precisión al final que se obtiene al redondear al valor más cercano.
model.compile(
    optimizer='rmsprop',
    loss='mse',
    metrics=['accuracy']
)

In [ ]:
#Se entrena el modelo con 1000 epochs y un early stopping en 100 epochs, con la capacidad de restaurar los mejores pesos
history = model.fit(
    x_train,
    y_train,
    epochs=1000,
    batch_size=1024,
    verbose=0,
    callbacks=[callbacks.EarlyStopping(monitor='val_loss', patience=100, min_delta=0, restore_best_weights=True)],
    validation_data=(x_val,y_val) #Se va validando el entrenamiento constantemente
)
model.save('perceptron.h5') #Se guarda el modelo para no tener que entrenarlo de nuevo

In [ ]:
import pickle
pickle.dump(history.history,open('history.pkl','wb'))

In [ ]:
history = pickle.load(open('history.pkl','rb'))

In [ ]:
#Se carga el modelo para realizar las pruebas
model = keras.models.load_model('perceptron.h5')

In [ ]:
#Se grafica el avance en la medida de loss con los datos de entrenamiento y validación
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Se grafica la mejora en el accuracy mientras entrena con los datos de entrenamiento y validación
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#Se realiza una predicción con la tercera porción de datos nunca vista por el modelo
y_pred = model.predict(x_test)

In [ ]:
#Se crea una función para graficar rápidamente los datos reales vs los datos predichos en la prueba
def graph_double(x1,x2):
    clear_output(wait=True)
    plt.plot(x1.index, x1)
    plt.plot(x1.index, x2)
    plt.legend(['real', 'pred'])
    coo = points.iloc[x1.name].values
    plt.title("x: {}, y: {}, z:{}".format(coo[0],coo[1],coo[2]))
    plt.show()

In [ ]:
#Se realiza una evaluación que arroja 95% de accuracy con los datos de prueba que nunca fueron vistos
model.evaluate(x_test,y_test)

In [ ]:
#Se visualiza la comparación entre los datos de prueba y la predicción del modelo
for i in range(1):
    graph_double(y_test.iloc[i],y_pred[i])

In [ ]:
#Se obtienen las coordenadas de los puntos en los ejes
un_points = points[2].unique()

In [ ]:
#Se evalúa el modelo con la totalidad de los datos
model.evaluate(points,bigdf)

In [ ]:
total_y = model.predict(points) #Se realiza una predicción de la totalidad de los datos

#Se obtiene una medida de error medio cuadrático para cada punto tomando en cuenta todos los instantes de tiempo
se = ((total_y - bigdf)**2)
mse = se.mean(axis=1)

In [ ]:
import seaborn as sns

In [ ]:
#Se grafica un mapa de calor del error medio cuadrático para cada nivel del eje z, en el plano x,y
for z in un_points:
    plotMx = np.zeros((len(un_points),len(un_points)))
    for i,x in enumerate(un_points):
        for j,y in enumerate(un_points):
            plotMx[-j,i] = mse[points[2] == z][points[0] == x][points[1] == y]
    plotDf = pd.DataFrame(plotMx)
    plotDf.columns = un_points
    plotDf.index = un_points*-1
    ax = sns.heatmap(plotDf,vmin=0, vmax=1e-5)
    ax.set_title("Error medio cuadrático en el plano x,y para z={}".format(z))
    plt.show()

In [ ]:
#Se visualizan los puntos originales en un gráfico
sel = (df_p[:,0] == 0.) * (df_p[:,1] == 0.) * (df_p[:,2] == 0.)

x1 = df.iloc[:,:300][sel].iloc[0]
x2 = model.predict(df_p)[sel].reshape(-1,1)
clear_output(wait=True)
plt.plot(x1.index, x1)
plt.plot(x1.index, x2)
plt.legend(['real', 'pred'])
coo = df_p[x1.name]
plt.title("x: {}, y: {}, z:{}".format(coo[0],coo[1],coo[2]))
plt.show()